In [3]:
!pip install -U huggingface_hub pillow

  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached click-8.3.1-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/2.9 MB ? eta -:--:--
   ---------------------------- ----------- 2.1/2.9 MB 14.7 MB/s eta 0:00:01
   ---------------------------------------- 2.9/2.9 MB 10.6 MB/s  0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---------------------------------- ----- 6.0/7.0 MB 28.4 MB/s eta 0:00:01
   ---------------------------------------- 7.0/7.0 MB 19.7 MB/s  0:00:00
Using cached fsspec-2025.12.0-py3-none-any.whl (201 kB)
Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
Using cached click-8.3.1-py3-none-any.whl (108 kB)

   ----- ---------------------------------- 1/8 [pillow]
   ----- ---------------------------------- 1/8 [pillow]
   ----- ---------------------------------- 1/8 [pillow]
   ----- -----------

In [ ]:
# CELL 1 — Imports + Config (HF + Ollama)

import os, time, json
import requests

from typing import TypedDict, List, Literal
from langgraph.graph import StateGraph, START, END
from langchain_ollama import ChatOllama

from IPython.display import display, Markdown, Image
from pprint import pprint

from dotenv import load_dotenv
load_dotenv()

from huggingface_hub import InferenceClient

# ---- Hugging Face token (set it in env as HF_TOKEN) ----
HF_TOKEN = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not HF_TOKEN:
    raise ValueError("HF_TOKEN not found in environment. Set it before running the notebook.")

HF_HEADERS = {"Authorization": f"Bearer {HF_TOKEN}"}

# Uses your existing HF_TOKEN / HUGGINGFACEHUB_API_TOKEN from .env
hf_client = InferenceClient(
    provider="fal-ai",          # text-to-image is supported on fal-ai
    api_key=HF_TOKEN,
)

# ---- HF Inference endpoints ----
# Text-to-image model (choose one you have access to / that fits your usage)
HF_T2I_MODEL = "stabilityai/stable-diffusion-2-1"
HF_T2I_URL   = f"https://api-inference.huggingface.co/models/{HF_T2I_MODEL}"

# Object detection model (DETR is a solid default)
HF_OD_MODEL  = "facebook/detr-resnet-50"
HF_OD_URL   = f"https://api-inference.huggingface.co/models/{HF_OD_MODEL}"

# ---- Output folder ----
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---- Ollama models ----
TEXT_MODEL  = "llama3.2"


c:\Users\harsh.raj\OneDrive - Aster DM Healthcare\Codes\LangGraph\Practical LangGraph\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# CELL 2 — State + Nodes (Prompt → Local SDXL → Object Detection → Ollama Summary)

import os
import torch
from diffusers import DiffusionPipeline
from typing import TypedDict, List

# ---- Load Diffusion Model ONCE (important) ----
# NOTE: You can switch device_map to "cuda" if GPU is available
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.bfloat16,
    device_map="cpu"
)

pipe.enable_attention_slicing()  # reduces memory footprint

# ---- Workflow State ----
class WorkflowState(TypedDict, total=False):
    topic: str
    image_prompt: str
    image_path: str
    objects: List[str]
    summary: str

# ---- Ollama LLMs ----
llm_prompt  = ChatOllama(model=TEXT_MODEL, temperature=0.5)
llm_summary = ChatOllama(model=TEXT_MODEL, temperature=0.2)

# -------------------------
# Node 1: Topic → Image Prompt
# -------------------------
def generate_image_prompt(state: WorkflowState) -> WorkflowState:
    topic = state["topic"]

    sys = (
        "You are an expert prompt engineer for photorealistic image generation. "
        "Return ONLY one final prompt. No explanations."
    )
    user = (
        f"Topic: {topic}\n\n"
        "Write one detailed photorealistic prompt including:\n"
        "- subject and environment\n"
        "- lighting and mood\n"
        "- camera / lens style\n"
        "- ultra-detailed, cinematic realism\n"
        "Avoid: text, watermarks, logos."
    )

    resp = llm_prompt.invoke([("system", sys), ("user", user)])
    return {"image_prompt": resp.content.strip()}

# -------------------------
# Node 2: Prompt → Image (LOCAL SDXL)
# -------------------------
def generate_image(state: WorkflowState) -> WorkflowState:
    prompt = state["image_prompt"]
    out_path = os.path.join(OUTPUT_DIR, "generated.png")

    image = pipe(prompt).images[0]
    image.save(out_path)

    return {"image_path": out_path}

# -------------------------
# Node 3: Image → Object Detection (HF DETR — FREE)
# -------------------------
def detect_objects(state: WorkflowState) -> WorkflowState:
    image_path = state["image_path"]

    with open(image_path, "rb") as f:
        img_bytes = f.read()

    r = requests.post(
        HF_OD_URL,
        headers=HF_HEADERS,
        data=img_bytes,
        timeout=120
    )

    r.raise_for_status()
    detections = r.json()

    labels = []
    for d in detections:
        label = d.get("label")
        if label:
            labels.append(label)

    # De-duplicate while preserving order
    seen, unique = set(), []
    for l in labels:
        if l not in seen:
            seen.add(l)
            unique.append(l)

    return {"objects": unique}

# -------------------------
# Node 4: Objects → Final Summary
# -------------------------
def summarize(state: WorkflowState) -> WorkflowState:
    topic = state["topic"]
    objects = state.get("objects", [])

    prompt = (
        f"Topic: {topic}\n"
        f"Detected objects: {objects}\n\n"
        "Write a crisp final summary in 5 bullets:\n"
        "- what the image likely depicts\n"
        "- key objects and their roles\n"
        "- scene and atmosphere\n"
        "- notable visual details\n"
    )

    resp = llm_summary.invoke(prompt)
    return {"summary": resp.content.strip()}


c:\Users\harsh.raj\OneDrive - Aster DM Healthcare\Codes\LangGraph\Practical LangGraph\myenv\Lib\site-packages\torch\amp\autocast_mode.py:270: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:38<00:00,  5.47s/it]


In [12]:
# CELL 3 — Graph & Edges (no notebook-specific tricks here)

graph = StateGraph(WorkflowState)

graph.add_node("generate_image_prompt", generate_image_prompt)
graph.add_node("generate_image", generate_image)
graph.add_node("detect_objects", detect_objects)
graph.add_node("summarize", summarize)

graph.add_edge(START, "generate_image_prompt")
graph.add_edge("generate_image_prompt", "generate_image")
graph.add_edge("generate_image", "detect_objects")
graph.add_edge("detect_objects", "summarize")
graph.add_edge("summarize", END)

workflow = graph.compile()


In [ ]:
# CELL 4 — Run + Pretty Output + Checkpoints (ipynb-friendly)

def checkpoint(msg: str):
    display(Markdown(f"✅ **Checkpoint:** {msg}"))

def show_output(initial_state: dict, final_state: dict):
    checkpoint("Rendering output started")

    display(Markdown("## 🟦 Input"))
    display(Markdown(f"**Topic:** `{initial_state.get('topic','')}`"))

    display(Markdown("## 🖼️ Generated Image"))
    img_path = final_state.get("image_path")
    if img_path and os.path.exists(img_path):
        checkpoint("Image found on disk — displaying it")
        display(Image(filename=img_path))
        display(Markdown(f"Saved at: `{img_path}`"))
    else:
        checkpoint("Image not found / not generated")
        display(Markdown("_Image not available (generation failed or file missing)._"))

    display(Markdown("## 🧩 Detected Objects"))
    objs = final_state.get("objects", [])
    if objs:
        checkpoint(f"Objects detected — count: {len(objs)}")
        display(Markdown("\n".join([f"- {o}" for o in objs])))
    else:
        checkpoint("No objects detected (or detection node did not populate state)")
        display(Markdown("_No objects detected._"))

    display(Markdown("## 📝 Summary"))
    summary = final_state.get("summary", "")
    if summary:
        checkpoint("Summary present — rendering it")
        display(Markdown(summary.replace("\n", "  \n")))
    else:
        checkpoint("No summary found in final_state")
        display(Markdown("_No summary._"))

    checkpoint("Rendering output completed")


# ---- Execute with execution checkpoints ----
checkpoint("About to invoke workflow")

initial_state = {"topic": "Bengal tiger walking through tall grass at sunrise, golden light, shallow depth"}
checkpoint("Initial state created")

final_state = workflow.invoke(initial_state)

checkpoint("Workflow invocation completed — now displaying results")
show_output(initial_state, final_state)

# Optional: also inspect raw keys/state
checkpoint("Displaying raw state keys")
display(Markdown("### 🔎 Raw state keys"))
pprint(list(final_state.keys()))
